In [23]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
from matplotlib import style

from scipy.stats import kurtosis, skew
import scipy.stats as stats

from math import sqrt

import statsmodels.formula.api as smf

In [24]:
df = pd.read_csv("cleaned_data_kc_house_data")

In [34]:
df = df.drop("id", axis = 1)

remove target variable<br>
also took out id since its only use is as an index

In [35]:
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,10/13/2014,221900,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,12/9/2014,538000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2/25/2015,180000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,12/9/2014,604000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,2/18/2015,510000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


will perform log on all continuous variables and not categorical

In [40]:
cont_log = pd.DataFrame([])
cont_log["sqft_living"] = np.log(df["sqft_living"])
cont_log["sqft_lot"] = np.log(df["sqft_lot"])
cont_log["sqft_above"] = np.log(df["sqft_above"])
cont_log["sqft_basement"] = np.log(df["sqft_basement"])
cont_log["yr_built"] = np.log(df["yr_built"])
cont_log["yr_renovated"] = np.log(df["yr_renovated"])

cont_log["sqft_living15"] = np.log(df["sqft_living15"])
cont_log["sqft_lot15"] = np.log(df["sqft_lot15"])
cont_log["price"] = np.log(df["price"])

C:\Users\mngav\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
C:\Users\mngav\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys


In [41]:
cont_log.describe()

,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,price
count,21082.000000,21082.000000,21082.000000,2.108200e+04,21082.000000,2.108200e+04,21082.000000,21082.000000,21082.000000
mean,7.550815,8.990663,7.395389,-inf,7.586197,-inf,7.539641,8.961429,13.048217
std,0.423980,0.901414,0.427009,NaN,0.014932,NaN,0.327421,0.811068,0.526394
min,5.913503,6.253829,5.913503,-inf,7.549609,-inf,5.988961,6.478510,11.264464
25%,7.265430,8.525161,7.090077,-inf,7.576610,-inf,7.306531,8.536996,12.682307
50%,7.554859,8.938532,7.352441,-inf,7.588324,-inf,7.517521,8.939319,13.017003
75%,7.843849,9.277789,7.700748,6.327937e+00,7.599401,-inf,7.766417,9.219176,13.377006
max,9.513404,14.317109,9.149528,8.480529e+00,7.608374,7.608374e+00,8.733916,13.677627,15.856731


In [42]:
cont_log.isna().sum()

sqft_living      0
sqft_lot         0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
sqft_living15    0
sqft_lot15       0
price            0
dtype: int64

too many variables for the scatter plots visualization, lets look at a correlation matrix

I'd like to look at individual correlations to help decide what features to drop. 

In [43]:
col_names = cont_log.describe().columns.drop([ "price"])

results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Homes: price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = smf.ols(formula=f, data=cont_log).fit()
    X_new = pd.DataFrame({val: [cont_log[val].min(), cont_log[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    print(results[idx+1])

Homes: price~sqft_living
------------------------------
['sqft_living', 0.45487390817074236, 6.725492042621481, 0.8373565737137351, 0.0]
Homes: price~sqft_lot
------------------------------
['sqft_lot', 0.01912643850674478, 12.32211855226208, 0.08076135917588545, 1.5829053476127593e-90]
Homes: price~sqft_above
------------------------------
['sqft_above', 0.34318242539406996, 7.7075256855493475, 0.7221650180792578, 0.0]
Homes: price~sqft_basement
------------------------------


MissingDataError: exog contains inf or nans

In [44]:
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.108200e+04,21082.000000,21082.000000,21082.000000,2.108200e+04,21082.00000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000,21082.000000
mean,5.402469e+05,3.372403,2.115916,2080.359975,1.507759e+04,1.49362,0.006688,0.233327,3.409828,7.657717,1789.000000,291.359975,1971.023337,68.359359,98077.858837,47.560371,-122.213843,1986.917418,12732.514135
std,3.667323e+05,0.924996,0.768142,917.856396,4.117338e+04,0.53937,0.081509,0.765066,0.650597,1.173690,828.369355,442.007858,29.323120,363.029573,53.528769,0.138568,0.140607,685.544250,27148.781580
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.00000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.00000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1952.000000,0.000000,98033.000000,47.471325,-122.328000,1490.000000,5100.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.620000e+03,1.50000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.572200,-122.230000,1840.000000,7626.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.069775e+04,2.00000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98117.000000,47.678200,-122.125000,2360.000000,10088.750000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.50000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [48]:
df.replace([np.inf, -np.inf], np.nan)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,10/13/2014,221900,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,12/9/2014,538000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2/25/2015,180000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,12/9/2014,604000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,2/18/2015,510000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,5/12/2014,1230000,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,4/15/2015,229500,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
7,3/12/2015,323000,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570
8,4/3/2015,662500,3,2.50,3560,9796,1.0,0,0,3,8,1860,1700,1965,0,98007,47.6007,-122.145,2210,8925
9,5/27/2014,468000,2,1.00,1160,6000,1.0,0,0,4,7,860,300,1942,0,98115,47.6900,-122.292,1330,6000


In [49]:
df.isna().sum()

date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [50]:
df.bathrooms.value_counts()

2.50    5242
1.00    3748
1.75    2978
2.25    2005
2.00    1882
1.50    1418
2.75    1160
3.00     735
3.50     718
3.25     570
3.75     152
4.00     135
4.50      96
4.25      77
0.75      71
4.75      23
5.00      19
5.25      13
1.25       9
5.50       9
6.00       5
5.75       4
0.50       3
8.00       2
6.25       2
6.75       2
6.50       2
7.50       1
7.75       1
Name: bathrooms, dtype: int64